<a href="https://colab.research.google.com/github/BadPrograms/ICA/blob/main/ica1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from numba import njit
import random
import matplotlib.pyplot as plt

scaler = StandardScaler(with_std=False)
@njit
def logistic(u):
    return 1/(1+np.exp(-u))
@njit
def log_dif(u):
  return (np.exp(u)-1)*sigmoid(u)

In [48]:
@njit
def sigmoid(X):
    return 1 / (1+np.exp(-X))
@njit
def unmixer(X):
    N, M = X.shape
    W = np.eye(M)+1
    W0=np.ones(M)
    print(W.shape)
    scale=8000
    X=X/scale
    anneal = [0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.02, 0.02, 0.01, 0.01,
                    0.005, 0.005, 0.002, 0.002, 0.001, 0.001, 0.001, 0.0005, 0.0005]
    print('Separating tracks ...')
    print(X.shape)
    W_temp = np.zeros_like(W)
    for alpha in anneal:
      for _ in range(50):

            x = X
            #print(x.shape)
            Z = 1 - 2 * sigmoid( (W @ x) +W0)
            W += alpha * (Z @ x.T + np.linalg.inv(W.T))
            #print((Z @ x.T + np.linalg.inv(W.T)))   
            W0 += alpha * (Z)   
    return W

In [66]:
@njit
def sigmoid(X):
    return 1 / (1+np.exp(-X))
@njit
def unmixer2(X):
    N, M = X.shape
    W = np.eye(M)+1
    print(W.shape)
    X=X
    anneal = [0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.02, 0.02, 0.01, 0.01,
                    0.005, 0.005, 0.002, 0.002, 0.001, 0.001, 0.001, 0.0005, 0.0005]
    print('Separating tracks ...')
    print(X.shape)
    W_temp = np.zeros_like(W)
    for alpha in anneal:
      for _ in range(90):

            x = X.T
            #print(x.shape)
            Z = np.eye(M)+(log_dif(W@x)@x.T)
            W += alpha * (Z @ W )
            #print((Z @ x.T + np.linalg.inv(W.T)))   
            
    return W

In [49]:
def whitening(X):
    cov = np.cov(X)
    d, E = np.linalg.eigh(cov)
    D = np.diag(d)
    D_inv = np.sqrt(np.linalg.inv(D))
    X_whiten = np.dot(E, np.dot(D_inv, np.dot(E.T, X)))
    return X_whiten

In [50]:
import PIL as pil
from google.colab import files
import sys
import io
import wave
uploaded = files.upload()
#np.set_printoptions(threshold=sys.maxsize)
import matplotlib.pyplot as plt

In [51]:
from IPython.display import Audio, display
from scipy.io import wavfile as wf

#display(Audio(uploaded["beet.wav"], autoplay=True))


In [52]:
_1,s1=wf.read("beet.wav")

In [53]:
_2,s2=wf.read("beet9.wav")
_3,s3=wf.read("mike.wav")
_4,s4=wf.read("beet92.wav")
_5,s5=wf.read("street.wav")
print(s3.shape)

(191258,)


In [54]:
display(Audio(s1,rate=_1, autoplay=True))
display(Audio(s2,rate=_2, autoplay=True))


In [55]:
x =np.stack((s3, s2[:s3.shape[0]],s1[:s3.shape[0]],s4[:s3.shape[0]],s5[:s3.shape[0]]))
A=np.random.rand(5,5)
print(A)
x=np.dot(A,x)
#print(f'Shape of s1: {s1.shape}, s2: {s1.shape}, Linear Mix: {x.shape}')
#wf.write('./talk_and_music.wav', _2, x.mean(axis=0).astype(np.float32))

[[0.16116411 0.22898883 0.52197118 0.70151472 0.9733946 ]
 [0.5232907  0.19100505 0.69240446 0.95496066 0.8737553 ]
 [0.18525867 0.5188173  0.78399072 0.7150618  0.0062392 ]
 [0.82851264 0.79784405 0.52562448 0.66071623 0.25799924]
 [0.23458282 0.82157617 0.10152206 0.61001206 0.22154529]]


In [56]:
print(A)

[[0.16116411 0.22898883 0.52197118 0.70151472 0.9733946 ]
 [0.5232907  0.19100505 0.69240446 0.95496066 0.8737553 ]
 [0.18525867 0.5188173  0.78399072 0.7150618  0.0062392 ]
 [0.82851264 0.79784405 0.52562448 0.66071623 0.25799924]
 [0.23458282 0.82157617 0.10152206 0.61001206 0.22154529]]


In [57]:
display(Audio(x[0],rate=_1, autoplay=True))
display(Audio(x[1],rate=_2, autoplay=True))

In [58]:
x_whiten=whitening(x)
print(x_whiten.shape)

(5, 191258)


In [59]:
display(Audio(x[0],rate=_1, autoplay=True))

In [67]:
a=unmixer2(x_whiten.T)

(5, 5)
Separating tracks ...
(191258, 5)


In [44]:
b=normalize(a@x)
print(a)
#b2=np.dot(a2,x)

[[2947.10717382 -230.69461232 -230.69461232 -230.69461232 -230.69461232]
 [-230.69461232 2947.10717382 -230.69461232 -230.69461232 -230.69461232]
 [-230.69461232 -230.69461232 2947.10717382 -230.69461232 -230.69461232]
 [-230.69461232 -230.69461232 -230.69461232 2947.10717382 -230.69461232]
 [-230.69461232 -230.69461232 -230.69461232 -230.69461232 2947.10717382]]


In [75]:
display(Audio(b[2],rate=_1, autoplay=True))
#display(Audio(x[1],rate=_1, autoplay=True))

In [46]:
print(normalize(np.dot(a,A)))

[[ 0.22952155  0.06611347  0.33193395  0.85227903  0.32617354]
 [ 0.64279968  0.15958437  0.03013106  0.02721852  0.7481261 ]
 [ 0.35533548 -0.05678862 -0.03432323  0.68600492  0.63145144]
 [ 0.09038607  0.96062054  0.12039064 -0.16438856  0.16589466]
 [ 0.18437356 -0.11338251  0.77282598  0.57003662 -0.17592343]]
